In [1]:
import numpy as np 
import sympy as sy
import matplotlib.pyplot as plt
import tensorflow as tf 
import random
from IPython.core.interactiveshell import InteractiveShell
import warnings 
InteractiveShell.ast_node_interactive = "all"
warnings.filterwarnings("ignore")

2022-03-24 07:01:19.285954: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 07:01:19.285969: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Theory

Consider a system where:

\begin{equation}
h^t= f (h^{t-1},\theta)  
\end{equation}
\begin{equation}
\theta \text{: parameters shared across all time steps}
\end{equation}

That is, its state at time step t, is dependent only on a set a parameters and the previous state at t-1
<br>
<br>
Let the state of the system, h, also be depedent on an input at the respective time step, x:

\begin{equation}
h^t= f (h^{t-1},x^{t},\theta)  
\end{equation}

The state h now contains information about the entire past history of inputs, x.

Consider now a system that given the hidden state, h,produces an output o, for each time step. This output is passed to an activation function made to predict the target, y, at the respective time step.

\begin{equation}
o^t= g (h^{t},\theta')  
\end{equation}
\begin{equation}
\theta' \text{: a different set of parameters as $\theta$}
\end{equation}


We define now define $\theta$ and $\theta'$ as the weight matrices describing the relation between the input-to-hidden, hidden-to-hidden and hidden-to-output notes; $U$, $W$ and $V$:

\begin{equation}
z^t= (h^{t-1})^{T} W + (x^t)^{T}U +b 
\end{equation}

\begin{equation}
h^{t} = \phi(z^t)
\end{equation}

\begin{equation}
o^t = (h^{t})^{T}V + c
\end{equation}

Then for each time step, we have a sequential total loss, L, defined as the difference between our prediction and the target
<br>
<br>
Consider the case of multi-class classification -> output activation function is the normalized expontential function, a.k.a the _softmax function_

\begin{equation}
L^T = \sum_{t=1}^{T} l(o^{t})
\text{: Total loss upto time step T}  
\end{equation}

\begin{equation}
\hat{y}^t_i = \frac{\exp(o_i^t)}{\sum_{j}\exp(o_j^t)}
\text{: Sigmoid activation function for multi-class classification}
\end{equation}

\begin{equation}
l(o^{t}) = \sum_{m=0}^{M-1}y_{m}^{t}(o^t) \log\Bigg(\frac{1}{\hat{y}_{m}^{t}(o^t)}\Bigg)
\text{: Categorical cross entropy}
\end{equation}